#**CELLA 1 - CONFIG AND LOAD DATA**

In [1]:
# ═══════════════════════════════════════════════════════════════════════════
# STATUS MODULE - MODELING (Model Comparison & Selection)
# ═══════════════════════════════════════════════════════════════════════════
#
# OBIETTIVO:
# Comparare 6 modelli per classificazione experience level, selezionare migliore
#
# INPUT:
#   - models/status_preprocessed_v2.2.pkl (X_train/test, y_train/test, scaler)
#
# OUTPUT:
#   - models/status_best_model_v2.2.pkl (best model trained)
#   - models/status_model_comparison_v2.2.json (performance metrics)
#   - visualizations/STATUS_Modeling_v2.2/ (confusion matrices, feature importance)
#
# MODELLI TESTATI:
#   1. Dummy Classifier (baseline assoluto)
#   2. Logistic Regression (linear baseline)
#   3. Decision Tree (interpretable, single tree)
#   4. Random Forest (bagging ensemble)
#   5. Gradient Boosting (sklearn boosting baseline)
#   6. XGBoost (candidate finale, regularized)
#
# METRICHE:
#   - Accuracy (test set)
#   - F1-macro, F1 per-class
#   - Train-test gap (overfitting check)
#   - Training time
#   - Feature importance (tree-based models)
#
# VERSIONE: 2.2
# DATA: 2026-02-09
# AUTORE: Alessandro Ambrosio
# ═══════════════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
import pickle
import json
import time
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report,
    confusion_matrix, ConfusionMatrixDisplay
)
from sklearn.model_selection import cross_val_score, StratifiedKFold

# XGBoost
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("STATUS MODULE - MODELING (Model Comparison)")
print("="*80)
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()





STATUS MODULE - MODELING (Model Comparison)
Timestamp: 2026-02-16 00:54:43



In [2]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 1: LOAD PREPROCESSED DATA
# ═══════════════════════════════════════════════════════════════════════════

print("="*80)
print("SECTION 1: LOAD PREPROCESSED DATA")
print("="*80)

# Paths
MODELDIR = Path('models')
VIZDIR = Path('visualizations/STATUS_Modeling_v2.2')
VIZDIR.mkdir(parents=True, exist_ok=True)

# Load preprocessed data
preprocessed_path = MODELDIR / 'status_preprocessed_v2.2.pkl'

print(f"\n[OK] Loading: {preprocessed_path}")

with open(preprocessed_path, 'rb') as f:
    data = pickle.load(f)

X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']
scaler = data['scaler']
feature_names = data['feature_names']
target_encoding = data['target_encoding']

print(f"\n[OK] X_train shape: {X_train.shape}")
print(f"[OK] X_test shape: {X_test.shape}")
print(f"[OK] Features: {len(feature_names)}")
print(f"[OK] Features: {feature_names}")

# Target distribution
print("\n" + "-"*80)
print("TARGET DISTRIBUTION")
print("-"*80)
print("\nTrain:")
print(y_train.value_counts().sort_index())
print("\nTest:")
print(y_test.value_counts().sort_index())

print()

SECTION 1: LOAD PREPROCESSED DATA

[OK] Loading: models/status_preprocessed_v2.2.pkl

[OK] X_train shape: (408, 7)
[OK] X_test shape: (102, 7)
[OK] Features: 7
[OK] Features: ['reps_mean', 'rpe_mean', 'total_sets', 'acwr_mean', 'spike_weeks_count', 'load_progression', 'skip_rate']

--------------------------------------------------------------------------------
TARGET DISTRIBUTION
--------------------------------------------------------------------------------

Train:
experience_label
Advanced        136
Beginner        136
Intermediate    136
Name: count, dtype: int64

Test:
experience_label
Advanced        34
Beginner        34
Intermediate    34
Name: count, dtype: int64



#**CELLA 3 - BASELINE MODELS**

In [3]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 2: BASELINE MODELS
# ═══════════════════════════════════════════════════════════════════════════

print("="*80)
print("SECTION 2: BASELINE MODELS")
print("="*80)

# Storage for results
results = []

# ────────────────────────────────────────────────────────────────────────────
# MODEL 1: Dummy Classifier (Stratified)
# ────────────────────────────────────────────────────────────────────────────

print("\n[1/6] Dummy Classifier (Stratified Baseline)...")

start_time = time.time()

dummy = DummyClassifier(strategy='stratified', random_state=42)
dummy.fit(X_train, y_train)

y_train_pred = dummy.predict(X_train)
y_test_pred = dummy.predict(X_test)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
train_f1 = f1_score(y_train, y_train_pred, average='macro')
test_f1 = f1_score(y_test, y_test_pred, average='macro')

elapsed = time.time() - start_time

print(f"  Train Accuracy: {train_acc:.3f}")
print(f"  Test Accuracy:  {test_acc:.3f}")
print(f"  Test F1-macro:  {test_f1:.3f}")
print(f"  Time: {elapsed:.2f}s")

results.append({
    'model': 'Dummy (Stratified)',
    'train_acc': train_acc,
    'test_acc': test_acc,
    'train_f1': train_f1,
    'test_f1': test_f1,
    'gap': train_acc - test_acc,
    'time_sec': elapsed
})

# ────────────────────────────────────────────────────────────────────────────
# MODEL 2: Logistic Regression
# ────────────────────────────────────────────────────────────────────────────

print("\n[2/6] Logistic Regression (Linear Baseline)...")

start_time = time.time()

logreg = LogisticRegression(
    max_iter=1000,
    random_state=42,
    solver='lbfgs',
    multi_class='multinomial'
)
logreg.fit(X_train, y_train)

y_train_pred = logreg.predict(X_train)
y_test_pred = logreg.predict(X_test)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
train_f1 = f1_score(y_train, y_train_pred, average='macro')
test_f1 = f1_score(y_test, y_test_pred, average='macro')

elapsed = time.time() - start_time

print(f"  Train Accuracy: {train_acc:.3f}")
print(f"  Test Accuracy:  {test_acc:.3f}")
print(f"  Test F1-macro:  {test_f1:.3f}")
print(f"  Gap: {train_acc - test_acc:.3f}")
print(f"  Time: {elapsed:.2f}s")

results.append({
    'model': 'Logistic Regression',
    'train_acc': train_acc,
    'test_acc': test_acc,
    'train_f1': train_f1,
    'test_f1': test_f1,
    'gap': train_acc - test_acc,
    'time_sec': elapsed
})

# ────────────────────────────────────────────────────────────────────────────
# MODEL 3: Decision Tree
# ────────────────────────────────────────────────────────────────────────────

print("\n[3/6] Decision Tree (Interpretable Baseline)...")

start_time = time.time()

dt = DecisionTreeClassifier(
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42
)
dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
train_f1 = f1_score(y_train, y_train_pred, average='macro')
test_f1 = f1_score(y_test, y_test_pred, average='macro')

elapsed = time.time() - start_time

print(f"  Train Accuracy: {train_acc:.3f}")
print(f"  Test Accuracy:  {test_acc:.3f}")
print(f"  Test F1-macro:  {test_f1:.3f}")
print(f"  Gap: {train_acc - test_acc:.3f}")
print(f"  Time: {elapsed:.2f}s")

results.append({
    'model': 'Decision Tree',
    'train_acc': train_acc,
    'test_acc': test_acc,
    'train_f1': train_f1,
    'test_f1': test_f1,
    'gap': train_acc - test_acc,
    'time_sec': elapsed
})

print()

SECTION 2: BASELINE MODELS

[1/6] Dummy Classifier (Stratified Baseline)...
  Train Accuracy: 0.368
  Test Accuracy:  0.294
  Test F1-macro:  0.295
  Time: 0.02s

[2/6] Logistic Regression (Linear Baseline)...
  Train Accuracy: 0.968
  Test Accuracy:  0.931
  Test F1-macro:  0.931
  Gap: 0.037
  Time: 0.12s

[3/6] Decision Tree (Interpretable Baseline)...
  Train Accuracy: 0.944
  Test Accuracy:  0.912
  Test F1-macro:  0.912
  Gap: 0.032
  Time: 0.06s



#**CELLA 4 - ENSEMBLE METHODS**

In [4]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 3: ENSEMBLE METHODS
# ═══════════════════════════════════════════════════════════════════════════

print("="*80)
print("SECTION 3: ENSEMBLE METHODS")
print("="*80)

# ────────────────────────────────────────────────────────────────────────────
# MODEL 4: Random Forest
# ────────────────────────────────────────────────────────────────────────────

print("\n[4/6] Random Forest (Bagging Ensemble)...")

start_time = time.time()

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=8,
    min_samples_split=10,
    min_samples_leaf=4,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
train_f1 = f1_score(y_train, y_train_pred, average='macro')
test_f1 = f1_score(y_test, y_test_pred, average='macro')

elapsed = time.time() - start_time

print(f"  Train Accuracy: {train_acc:.3f}")
print(f"  Test Accuracy:  {test_acc:.3f}")
print(f"  Test F1-macro:  {test_f1:.3f}")
print(f"  Gap: {train_acc - test_acc:.3f}")
print(f"  Time: {elapsed:.2f}s")

results.append({
    'model': 'Random Forest',
    'train_acc': train_acc,
    'test_acc': test_acc,
    'train_f1': train_f1,
    'test_f1': test_f1,
    'gap': train_acc - test_acc,
    'time_sec': elapsed
})

# ────────────────────────────────────────────────────────────────────────────
# MODEL 5: Gradient Boosting (sklearn)
# ────────────────────────────────────────────────────────────────────────────

print("\n[5/6] Gradient Boosting (sklearn baseline)...")

start_time = time.time()

gb = GradientBoostingClassifier(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=4,
    min_samples_split=10,
    min_samples_leaf=4,
    subsample=0.8,
    random_state=42
)
gb.fit(X_train, y_train)

y_train_pred = gb.predict(X_train)
y_test_pred = gb.predict(X_test)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
train_f1 = f1_score(y_train, y_train_pred, average='macro')
test_f1 = f1_score(y_test, y_test_pred, average='macro')

elapsed = time.time() - start_time

print(f"  Train Accuracy: {train_acc:.3f}")
print(f"  Test Accuracy:  {test_acc:.3f}")
print(f"  Test F1-macro:  {test_f1:.3f}")
print(f"  Gap: {train_acc - test_acc:.3f}")
print(f"  Time: {elapsed:.2f}s")

results.append({
    'model': 'Gradient Boosting',
    'train_acc': train_acc,
    'test_acc': test_acc,
    'train_f1': train_f1,
    'test_f1': test_f1,
    'gap': train_acc - test_acc,
    'time_sec': elapsed
})

# ────────────────────────────────────────────────────────────────────────────
# MODEL 6: XGBoost (Candidate Finale)
# ────────────────────────────────────────────────────────────────────────────

print("\n[6/6] XGBoost (Regularized, Candidate Finale)...")

# XGBoost requires numeric labels
label_encoder = {'Beginner': 0, 'Intermediate': 1, 'Advanced': 2}
y_train_encoded = y_train.map(label_encoder)
y_test_encoded = y_test.map(label_encoder)

start_time = time.time()

xgb = XGBClassifier(
    n_estimators=150,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,          # L1 regularization
    reg_lambda=1.0,         # L2 regularization
    random_state=42,
    n_jobs=-1,
    eval_metric='mlogloss'
)
xgb.fit(X_train, y_train_encoded)

y_train_pred_encoded = xgb.predict(X_train)
y_test_pred_encoded = xgb.predict(X_test)

# Decode back to string labels for consistency
reverse_encoder = {0: 'Beginner', 1: 'Intermediate', 2: 'Advanced'}
y_train_pred = pd.Series(y_train_pred_encoded).map(reverse_encoder)
y_test_pred = pd.Series(y_test_pred_encoded).map(reverse_encoder)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
train_f1 = f1_score(y_train, y_train_pred, average='macro')
test_f1 = f1_score(y_test, y_test_pred, average='macro')

elapsed = time.time() - start_time

print(f"  Train Accuracy: {train_acc:.3f}")
print(f"  Test Accuracy:  {test_acc:.3f}")
print(f"  Test F1-macro:  {test_f1:.3f}")
print(f"  Gap: {train_acc - test_acc:.3f}")
print(f"  Time: {elapsed:.2f}s")

results.append({
    'model': 'XGBoost',
    'train_acc': train_acc,
    'test_acc': test_acc,
    'train_f1': train_f1,
    'test_f1': test_f1,
    'gap': train_acc - test_acc,
    'time_sec': elapsed
})

print()


SECTION 3: ENSEMBLE METHODS

[4/6] Random Forest (Bagging Ensemble)...
  Train Accuracy: 0.985
  Test Accuracy:  0.941
  Test F1-macro:  0.941
  Gap: 0.044
  Time: 0.87s

[5/6] Gradient Boosting (sklearn baseline)...
  Train Accuracy: 1.000
  Test Accuracy:  0.951
  Test F1-macro:  0.951
  Gap: 0.049
  Time: 1.49s

[6/6] XGBoost (Regularized, Candidate Finale)...
  Train Accuracy: 1.000
  Test Accuracy:  0.931
  Test F1-macro:  0.931
  Gap: 0.069
  Time: 0.25s



#**CELLA 5 - MODEL COMPARISON (Extended Metrics)**

In [5]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 4: MODEL COMPARISON (Extended Metrics)
# ═══════════════════════════════════════════════════════════════════════════

print("="*80)
print("SECTION 4: MODEL COMPARISON (Extended Metrics)")
print("="*80)

from sklearn.metrics import precision_score, recall_score

# Re-train all models and collect extended metrics
models_dict = {
    'Dummy (Stratified)': DummyClassifier(strategy='stratified', random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42, solver='lbfgs', multi_class='multinomial'),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=200, max_depth=8, min_samples_split=10, min_samples_leaf=4, max_features='sqrt', random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=150, learning_rate=0.1, max_depth=4, min_samples_split=10, min_samples_leaf=4, subsample=0.8, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=150, max_depth=4, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.1, reg_lambda=1.0, random_state=42, n_jobs=-1, eval_metric='mlogloss')
}

# Extended results storage
extended_results = []

print("\nTraining models with extended metrics...")
print("-"*80)

for model_name, model in models_dict.items():
    print(f"\n{model_name}...")

    start_time = time.time()

    # Handle XGBoost encoding
    if model_name == 'XGBoost':
        label_encoder = {'Beginner': 0, 'Intermediate': 1, 'Advanced': 2}
        y_train_encoded = y_train.map(label_encoder)
        y_test_encoded = y_test.map(label_encoder)

        model.fit(X_train, y_train_encoded)

        y_train_pred_encoded = model.predict(X_train)
        y_test_pred_encoded = model.predict(X_test)

        reverse_encoder = {0: 'Beginner', 1: 'Intermediate', 2: 'Advanced'}
        y_train_pred = pd.Series(y_train_pred_encoded).map(reverse_encoder).values
        y_test_pred = pd.Series(y_test_pred_encoded).map(reverse_encoder).values
    else:
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

    elapsed = time.time() - start_time

    # Calculate metrics
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    train_f1 = f1_score(y_train, y_train_pred, average='macro')
    test_f1 = f1_score(y_test, y_test_pred, average='macro')

    train_precision = precision_score(y_train, y_train_pred, average='macro')
    test_precision = precision_score(y_test, y_test_pred, average='macro')

    train_recall = recall_score(y_train, y_train_pred, average='macro')
    test_recall = recall_score(y_test, y_test_pred, average='macro')

    # Per-class metrics (test set only)
    test_precision_per_class = precision_score(y_test, y_test_pred, average=None, labels=['Beginner', 'Intermediate', 'Advanced'])
    test_recall_per_class = recall_score(y_test, y_test_pred, average=None, labels=['Beginner', 'Intermediate', 'Advanced'])
    test_f1_per_class = f1_score(y_test, y_test_pred, average=None, labels=['Beginner', 'Intermediate', 'Advanced'])

    print(f"  Test Accuracy:  {test_acc:.3f}")
    print(f"  Test F1-macro:  {test_f1:.3f}")
    print(f"  Test Precision: {test_precision:.3f}")
    print(f"  Test Recall:    {test_recall:.3f}")
    print(f"  Train-Test Gap: {train_acc - test_acc:.3f}")
    print(f"  Time: {elapsed:.2f}s")

    extended_results.append({
        'model': model_name,
        'train_acc': train_acc,
        'test_acc': test_acc,
        'train_f1': train_f1,
        'test_f1': test_f1,
        'train_precision': train_precision,
        'test_precision': test_precision,
        'train_recall': train_recall,
        'test_recall': test_recall,
        'gap': train_acc - test_acc,
        'time_sec': elapsed,
        'test_precision_beginner': test_precision_per_class[0],
        'test_precision_intermediate': test_precision_per_class[1],
        'test_precision_advanced': test_precision_per_class[2],
        'test_recall_beginner': test_recall_per_class[0],
        'test_recall_intermediate': test_recall_per_class[1],
        'test_recall_advanced': test_recall_per_class[2],
        'test_f1_beginner': test_f1_per_class[0],
        'test_f1_intermediate': test_f1_per_class[1],
        'test_f1_advanced': test_f1_per_class[2]
    })

# Convert to DataFrame
df_results = pd.DataFrame(extended_results)

print("\n" + "="*80)
print("MODEL COMPARISON TABLE (Test Set)")
print("="*80)

# Display main metrics
print("\nOverall Performance:")
print(df_results[['model', 'test_acc', 'test_f1', 'test_precision', 'test_recall', 'gap', 'time_sec']].to_string(index=False))

print("\n" + "-"*80)
print("Per-Class Performance (Test Set)")
print("-"*80)

# Display per-class metrics
print("\nPrecision per class:")
print(df_results[['model', 'test_precision_beginner', 'test_precision_intermediate', 'test_precision_advanced']].to_string(index=False))

print("\nRecall per class:")
print(df_results[['model', 'test_recall_beginner', 'test_recall_intermediate', 'test_recall_advanced']].to_string(index=False))

print("\nF1-Score per class:")
print(df_results[['model', 'test_f1_beginner', 'test_f1_intermediate', 'test_f1_advanced']].to_string(index=False))

# Identify best model (by test F1-macro, considering gap < 0.10)
df_filtered = df_results[df_results['gap'] < 0.10]  # Filter overfitting models
if len(df_filtered) > 0:
    best_model_name = df_filtered.loc[df_filtered['test_f1'].idxmax(), 'model']
else:
    best_model_name = df_results.loc[df_results['test_f1'].idxmax(), 'model']

print("\n" + "="*80)
print(f"BEST MODEL: {best_model_name}")
print("="*80)
print(f"Selection criteria: Highest F1-macro with train-test gap < 0.10")

# Save comparison results
comparison_path = MODELDIR / 'status_model_comparison_v2.2.json'
df_results.to_json(comparison_path, orient='records', indent=2)
print(f"\n[OK] Comparison saved: {comparison_path}")

print()


SECTION 4: MODEL COMPARISON (Extended Metrics)

Training models with extended metrics...
--------------------------------------------------------------------------------

Dummy (Stratified)...
  Test Accuracy:  0.294
  Test F1-macro:  0.295
  Test Precision: 0.297
  Test Recall:    0.294
  Train-Test Gap: 0.074
  Time: 0.00s

Logistic Regression...
  Test Accuracy:  0.931
  Test F1-macro:  0.931
  Test Precision: 0.932
  Test Recall:    0.931
  Train-Test Gap: 0.037
  Time: 0.01s

Decision Tree...
  Test Accuracy:  0.912
  Test F1-macro:  0.912
  Test Precision: 0.914
  Test Recall:    0.912
  Train-Test Gap: 0.032
  Time: 0.01s

Random Forest...
  Test Accuracy:  0.941
  Test F1-macro:  0.941
  Test Precision: 0.942
  Test Recall:    0.941
  Train-Test Gap: 0.044
  Time: 0.76s

Gradient Boosting...
  Test Accuracy:  0.951
  Test F1-macro:  0.951
  Test Precision: 0.951
  Test Recall:    0.951
  Train-Test Gap: 0.049
  Time: 1.33s

XGBoost...
  Test Accuracy:  0.931
  Test F1-macro:  0

#**CELLA 6 - VISUALIZATIONS**

In [6]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 5: VISUALIZATIONS
# ═══════════════════════════════════════════════════════════════════════════

print("="*80)
print("SECTION 5: VISUALIZATIONS")
print("="*80)

# ────────────────────────────────────────────────────────────────────────────
# STATUS Brand Colors
# ────────────────────────────────────────────────────────────────────────────

STATUS_COLORS = {
    'navy': '#2B4162',
    'royal_blue': '#385F8F',
    'purple': '#7B5E9D',
    'light_purple': '#9B7EBD',
    'text_dark': '#1A1A1A',
    'text_light': '#FFFFFF'
}

STATUS_PALETTE = ['#2B4162', '#7B5E9D', '#9B7EBD']  # Beginner, Intermediate, Advanced

# Custom colormap (blue-purple gradient)
from matplotlib.colors import LinearSegmentedColormap
status_cmap = LinearSegmentedColormap.from_list(
    'status',
    ['#E8EAF6', '#9B7EBD', '#7B5E9D', '#385F8F', '#2B4162']
)

# Set seaborn style
sns.set_style("whitegrid")
sns.set_palette(STATUS_PALETTE)

# Re-train best model (Gradient Boosting) for visualization
print("\nRe-training best model (Gradient Boosting) for analysis...")

gb_best = GradientBoostingClassifier(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=4,
    min_samples_split=10,
    min_samples_leaf=4,
    subsample=0.8,
    random_state=42
)
gb_best.fit(X_train, y_train)
y_test_pred = gb_best.predict(X_test)

# ────────────────────────────────────────────────────────────────────────────
# 5.1 Confusion Matrix (STATUS Branding, No Grid)
# ────────────────────────────────────────────────────────────────────────────

print("\n[1/3] Confusion Matrix (STATUS Branding)...")

# ════════════════════════════════════════════════════════════
# NUMERICAL CONFUSION MATRIX ANALYSIS
# ════════════════════════════════════════════════════════════

print("\n" + "-"*80)
print("CONFUSION MATRIX - NUMERICAL ANALYSIS")
print("-"*80)

cm = confusion_matrix(y_test, y_test_pred,
                      labels=['Beginner', 'Intermediate', 'Advanced'])

classes = ['Beginner', 'Intermediate', 'Advanced']

print("\nRaw Confusion Matrix:")
print(pd.DataFrame(cm, index=classes, columns=classes))

accuracy = np.trace(cm) / np.sum(cm)
print(f"\nOverall Accuracy: {accuracy:.4f}")

# Per-class metrics
print("\nPer-Class Metrics:")
for i, cls in enumerate(classes):
    tp = cm[i, i]
    fn = cm[i, :].sum() - tp
    fp = cm[:, i].sum() - tp
    tn = cm.sum() - (tp + fn + fp)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\n{cls}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall   : {recall:.4f}")
    print(f"  F1-Score : {f1:.4f}")


fig, ax = plt.subplots(figsize=(9, 7))

cm = confusion_matrix(y_test, y_test_pred, labels=['Beginner', 'Intermediate', 'Advanced'])

# Custom colormap for confusion matrix
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=['Beginner', 'Intermediate', 'Advanced']
)
disp.plot(cmap=status_cmap, ax=ax, values_format='d', colorbar=False)

ax.grid(False)

# Customize title and labels
ax.set_title('STATUS Module - Confusion Matrix\nGradient Boosting (Test Set)',
             fontsize=16, weight='bold', color=STATUS_COLORS['navy'], pad=20)
ax.set_xlabel('Predicted Experience Level', fontsize=13, weight='bold', color=STATUS_COLORS['navy'])
ax.set_ylabel('True Experience Level', fontsize=13, weight='bold', color=STATUS_COLORS['navy'])

# Add accuracy annotation
accuracy = np.trace(cm) / np.sum(cm)
ax.text(0.5, -0.15, f'Accuracy: {accuracy:.1%}',
        transform=ax.transAxes, ha='center', fontsize=12,
        weight='bold', color=STATUS_COLORS['purple'])

plt.tight_layout()

cm_path = VIZDIR / 'confusion_matrix_best_model_v2.2.png'
plt.savefig(cm_path, dpi=300, bbox_inches='tight', facecolor='white')
print(f"  [OK] Saved: {cm_path}")
plt.close()


# ────────────────────────────────────────────────────────────────────────────
# 5.2 Feature Importance
# ────────────────────────────────────────────────────────────────────────────

print("\n[2/3] Feature Importance...")

feature_importance = gb_best.feature_importances_
feature_df = (pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)
  .reset_index(drop=True))

# ════════════════════════════════════════════════════════════
# NUMERICAL FEATURE IMPORTANCE ANALYSIS
# ════════════════════════════════════════════════════════════

print("\n" + "-"*80)
print("FEATURE IMPORTANCE - NUMERICAL RANKING")
print("-"*80)

feature_df = feature_df.reset_index(drop=True)

total_importance = feature_df['importance'].sum()

for i, row in feature_df.iterrows():
    perc = (row['importance'] / total_importance) * 100
    print(f"{i+1:2d}. {row['feature']:25s} "
          f"| Importance: {row['importance']:.4f} "
          f"| Contribution: {perc:.2f}%")

print("\nCumulative Importance (Top Features):")
feature_df['cumulative'] = feature_df['importance'].cumsum()

for i in range(len(feature_df)):
    print(f"Top {i+1:2d} → {feature_df.loc[i, 'cumulative']:.4f}")

# Dominance check
top3 = feature_df.head(3)['importance'].sum()
print(f"\nTop 3 features contribution: {(top3/total_importance)*100:.2f}%")

if top3/total_importance > 0.70:
    print("[WARNING] Model heavily driven by few features")

fig, ax = plt.subplots(figsize=(11, 7))

# Gradient colors (most important = darkest)
n_features = len(feature_df)
colors = [STATUS_COLORS['navy'] if i == 0
          else STATUS_COLORS['royal_blue'] if i == 1
          else STATUS_COLORS['purple'] if i == 2
          else STATUS_COLORS['light_purple']
          for i in range(n_features)]

bars = ax.barh(feature_df['feature'], feature_df['importance'], color=colors, edgecolor='white', linewidth=1.5)

ax.set_xlabel('Importance Score', fontsize=13, weight='bold', color=STATUS_COLORS['navy'])
ax.set_title('STATUS Module - Feature Importance\nGradient Boosting Classifier',
             fontsize=16, weight='bold', color=STATUS_COLORS['navy'], pad=20)
ax.invert_yaxis()
ax.set_xlim([0, max(feature_df['importance']) * 1.15])

# Add value labels
for i, (bar, val) in enumerate(zip(bars, feature_df['importance'])):
    width = bar.get_width()
    ax.text(width + 0.01, bar.get_y() + bar.get_height()/2,
            f'{val:.3f}',
            ha='left', va='center', fontsize=11, weight='bold',
            color=STATUS_COLORS['navy'])

# Add ranking numbers
for i, bar in enumerate(bars):
    ax.text(0.685, bar.get_y() + bar.get_height()/2,
            f'#{i+1}',
            ha='right', va='center', fontsize=10, weight='bold',
            color=STATUS_COLORS['purple'])

# Grid styling
ax.grid(axis='x', alpha=0.3, linestyle='--', color=STATUS_COLORS['royal_blue'])
ax.set_axisbelow(True)

plt.tight_layout()

fi_path = VIZDIR / 'feature_importance_best_model_v2.2.png'
plt.savefig(fi_path, dpi=300, bbox_inches='tight', facecolor='white')
print(f"  [OK] Saved: {fi_path}")
plt.close()

print("\nTop 3 Features:")
for i, row in feature_df.head(3).iterrows():
    print(f"  {i+1}. {row['feature']:20s}: {row['importance']:.3f}")


# ────────────────────────────────────────────────────────────────────────────
# 5.3 Model Comparison Bar Chart
# ────────────────────────────────────────────────────────────────────────────

print("\n[3/3] Model Comparison Chart...")

# ════════════════════════════════════════════════════════════
# NUMERICAL MODEL COMPARISON
# ════════════════════════════════════════════════════════════

print("\n" + "-"*80)
print("MODEL COMPARISON - NUMERICAL ANALYSIS")
print("-"*80)

models_to_plot = df_results[df_results['model'] != 'Dummy (Stratified)'].copy()
models_to_plot = models_to_plot.sort_values('test_acc', ascending=False)

for _, row in models_to_plot.iterrows():

    gap = row['train_acc'] - row['test_acc']

    print(f"\nModel: {row['model']}")
    print(f"  Train Accuracy : {row['train_acc']:.4f}")
    print(f"  Test Accuracy  : {row['test_acc']:.4f}")
    print(f"  Generalization Gap: {gap:.4f}")

    if gap >= 0.10:
        print("  [OVERFITTING RISK]")
    elif gap >= 0.05:
        print("  [MILD GAP]")
    else:
        print("  [GOOD GENERALIZATION]")


best_model = models_to_plot.iloc[0]
print("\n" + "-"*40)
print(f"BEST MODEL: {best_model['model']}")
print(f"Test Accuracy: {best_model['test_acc']:.4f}")
print("-"*40)


fig, ax = plt.subplots(figsize=(12, 7))

models_to_plot = df_results[df_results['model'] != 'Dummy (Stratified)'].copy()
models_to_plot = models_to_plot.sort_values('test_f1', ascending=False)
best_model = models_to_plot.iloc[0]

y_pos = np.arange(len(models_to_plot))

# Train accuracy (background, lighter)
bars_train = ax.barh(y_pos, models_to_plot['train_acc'],
                      color=STATUS_COLORS['light_purple'], alpha=0.4,
                      label='Train Accuracy', edgecolor='white', linewidth=1.5)

# Test accuracy (foreground, darker)
bars_test = ax.barh(y_pos, models_to_plot['test_acc'],
                     color=STATUS_COLORS['navy'], alpha=0.85,
                     label='Test Accuracy', edgecolor='white', linewidth=1.5)

ax.set_yticks(y_pos)
ax.set_yticklabels(models_to_plot['model'], fontsize=11, weight='bold')
ax.set_xlabel('Accuracy', fontsize=13, weight='bold', color=STATUS_COLORS['navy'])
ax.set_title('STATUS Module - Model Comparison\nTrain vs Test Accuracy',
             fontsize=16, weight='bold', color=STATUS_COLORS['navy'], pad=20)
ax.legend(loc='lower right', fontsize=11, framealpha=0.95)
ax.set_xlim([0.85, 1.02])

# Add value labels (test accuracy only)
for i, test in enumerate(models_to_plot['test_acc']):
    ax.text(test + 0.003, i, f'{test:.1%}',
            va='center', fontsize=11, weight='bold',
            color=STATUS_COLORS['navy'])

# Highlight best model
best_idx = models_to_plot['test_acc'].idxmax()
best_pos = np.where(models_to_plot.index == best_idx)[0][0]
ax.axhline(best_pos, color=STATUS_COLORS['purple'], linewidth=3, alpha=0.3, linestyle='--')

# Add "BEST" annotation
ax.text(0.99, best_pos, '  ★ BEST', va='center', ha='right',
        fontsize=10, weight='bold', color=STATUS_COLORS['purple'],
        bbox=dict(boxstyle='round,pad=0.3', facecolor=STATUS_COLORS['light_purple'],
                  alpha=0.3, edgecolor=STATUS_COLORS['purple']))

# Grid styling
ax.grid(axis='x', alpha=0.3, linestyle='--', color=STATUS_COLORS['royal_blue'])
ax.set_axisbelow(True)

plt.tight_layout()

comp_path = VIZDIR / 'model_comparison_v2.2.png'
plt.savefig(comp_path, dpi=300, bbox_inches='tight', facecolor='white')
print(f"  [OK] Saved:  {comp_path}")
plt.close()

print()


SECTION 5: VISUALIZATIONS

Re-training best model (Gradient Boosting) for analysis...

[1/3] Confusion Matrix (STATUS Branding)...

--------------------------------------------------------------------------------
CONFUSION MATRIX - NUMERICAL ANALYSIS
--------------------------------------------------------------------------------

Raw Confusion Matrix:
              Beginner  Intermediate  Advanced
Beginner            34             0         0
Intermediate         1            32         1
Advanced             0             3        31

Overall Accuracy: 0.9510

Per-Class Metrics:

Beginner
  Precision: 0.9714
  Recall   : 1.0000
  F1-Score : 0.9855

Intermediate
  Precision: 0.9143
  Recall   : 0.9412
  F1-Score : 0.9275

Advanced
  Precision: 0.9688
  Recall   : 0.9118
  F1-Score : 0.9394
  [OK] Saved: visualizations/STATUS_Modeling_v2.2/confusion_matrix_best_model_v2.2.png

[2/3] Feature Importance...

--------------------------------------------------------------------------------

#**CELLA 7 - MODEL SELECTION RATIONALE & SAVE**

In [7]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 6: MODEL SELECTION RATIONALE & SAVE
# ═══════════════════════════════════════════════════════════════════════════

print("="*80)
print("SECTION 6: MODEL SELECTION RATIONALE & SAVE")
print("="*80)

from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score, classification_report
)

# --- Metrics + report ---
y_train_pred = gb_best.predict(X_train)
y_test_pred  = gb_best.predict(X_test)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1_macro = f1_score(y_test, y_test_pred, average='macro')
test_precision_macro = precision_score(y_test, y_test_pred, average='macro')
test_recall_macro = recall_score(y_test, y_test_pred, average='macro')

train_accuracy = accuracy_score(y_train, y_train_pred)
train_test_gap = train_accuracy - test_accuracy

report = classification_report(y_test, y_test_pred, output_dict=True)

# Per-class metrics (from report)
beg_prec = report['Beginner']['precision']
beg_rec  = report['Beginner']['recall']
beg_f1   = report['Beginner']['f1-score']

int_prec = report['Intermediate']['precision']
int_rec  = report['Intermediate']['recall']
int_f1   = report['Intermediate']['f1-score']

adv_prec = report['Advanced']['precision']
adv_rec  = report['Advanced']['recall']
adv_f1   = report['Advanced']['f1-score']

print("\n" + "-"*80)
print("BEST MODEL: Gradient Boosting")
print("-"*80)

print("\nWhy Gradient Boosting over alternatives:")
print(f"[OK] Test accuracy: {test_accuracy:.1%}")
print(f"[OK] Test F1-macro: {test_f1_macro:.3f}")
print(f"[OK] Beginner recall: {beg_rec:.1%}")
print(f"[OK] Advanced precision: {adv_prec:.1%}")
print(f"[OK] Train-test gap: {train_test_gap:.3f} (< 0.10 threshold)")
print("[OK] Handles non-linear interactions (skip_rate × total_sets)")

def row_of(name: str):
    return df_results.loc[df_results['model'].eq(name)].iloc[0]

lr = row_of('Logistic Regression')
xg = row_of('XGBoost')

print("\nWhy NOT Logistic Regression:")
print(f"[X] Lower test accuracy: {lr['test_acc']:.1%} vs {test_accuracy:.1%}")
print(f"[OK] Better gap: {lr['gap']:.3f} vs {train_test_gap:.3f}")

print("\nWhy NOT XGBoost:")
print(f"[X] Larger gap: {xg['gap']:.3f} (threshold 0.10)")
print(f"[X] Lower test accuracy: {xg['test_acc']:.1%} vs {test_accuracy:.1%}")

# --- Save best model bundle (PKL) ---
print("\n" + "-"*80)
print("SAVING BEST MODEL")
print("-"*80)

best_model_bundle = {
    'model': gb_best,
    'scaler': scaler,
    'feature_names': feature_names,
    'target_encoding': target_encoding,
    'model_name': 'Gradient Boosting',
    'test_accuracy': float(test_accuracy),
    'test_f1_macro': float(test_f1_macro),
    'test_precision_macro': float(test_precision_macro),
    'test_recall_macro': float(test_recall_macro),
    'train_test_gap': float(train_test_gap),
    'hyperparameters': gb_best.get_params(),
    'feature_importance': feature_df.to_dict('records'),
    'version': '2.2',
    'timestamp': datetime.now().isoformat()
}

model_path = MODELDIR / 'status_best_model_v2.2.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(best_model_bundle, f)

print(f"\n[OK] Best model saved: {model_path}")
print(f"  File size: {model_path.stat().st_size / 1024:.1f} KB")

# --- Save model info (JSON) ---
model_info = {
    'model_name': 'Gradient Boosting',
    'version': '2.2',
    'date_trained': datetime.now().isoformat(),
    'dataset': {
        'n_train': int(len(X_train)),
        'n_test': int(len(X_test)),
        'n_features': int(len(feature_names)),
        'features': feature_names
    },
    'performance': {
        'test_accuracy': float(test_accuracy),
        'test_f1_macro': float(test_f1_macro),
        'test_precision_macro': float(test_precision_macro),
        'test_recall_macro': float(test_recall_macro),
        'train_test_gap': float(train_test_gap),
        'classification_report': report
    },
    'hyperparameters': gb_best.get_params(),
    'feature_importance_top3': feature_df.head(3).to_dict('records')
}

info_path = MODELDIR / 'status_model_info_v2.2.json'
with open(info_path, 'w') as f:
    json.dump(model_info, f, indent=2, default=float)  # robust vs numpy types [web:10][web:575]

print(f"[OK] Model info saved: {info_path}")


SECTION 6: MODEL SELECTION RATIONALE & SAVE

--------------------------------------------------------------------------------
BEST MODEL: Gradient Boosting
--------------------------------------------------------------------------------

Why Gradient Boosting over alternatives:
[OK] Test accuracy: 95.1%
[OK] Test F1-macro: 0.951
[OK] Beginner recall: 100.0%
[OK] Advanced precision: 96.9%
[OK] Train-test gap: 0.049 (< 0.10 threshold)
[OK] Handles non-linear interactions (skip_rate × total_sets)

Why NOT Logistic Regression:
[X] Lower test accuracy: 93.1% vs 95.1%
[OK] Better gap: 0.037 vs 0.049

Why NOT XGBoost:
[X] Larger gap: 0.069 (threshold 0.10)
[X] Lower test accuracy: 93.1% vs 95.1%

--------------------------------------------------------------------------------
SAVING BEST MODEL
--------------------------------------------------------------------------------

[OK] Best model saved: models/status_best_model_v2.2.pkl
  File size: 729.5 KB
[OK] Model info saved: models/status_mode

#**CELLA 8 - SUMMARY**

In [8]:
# ═══════════════════════════════════════════════════════════════════════════
# SECTION 7: SUMMARY
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("STATUS MODELING v2.2 - COMPLETE")
print("="*80)

print("\n┌" + "─"*78 + "┐")
print("│" + " "*26 + "MODELING COMPLETE" + " "*35 + "│")
print("└" + "─"*78 + "┘")

print("\nBEST MODEL")
print("-"*80)
print("Model:           Gradient Boosting")
print(f"Test Accuracy:   {test_accuracy:.1%}")
print(f"F1-macro:        {test_f1_macro:.3f}")
print(f"Train-Test Gap:  {train_test_gap:.3f} (< 0.10 [OK])")

print("\nPER-CLASS PERFORMANCE")
print("-"*80)
print(f"Beginner:        Precision {beg_prec:.1%}, Recall {beg_rec:.1%}, F1 {beg_f1:.1%}")
print(f"Intermediate:    Precision {int_prec:.1%}, Recall {int_rec:.1%}, F1 {int_f1:.1%}")
print(f"Advanced:        Precision {adv_prec:.1%}, Recall {adv_rec:.1%}, F1 {adv_f1:.1%}")

print("\nOUTPUT FILES")
print("-"*80)
print(f"[OK] {model_path}")
print(f"[OK] {info_path}")
print(f"[OK] {comparison_path}")
print(f"[OK] {cm_path}")
print(f"[OK] {fi_path}")
print(f"[OK] {comp_path}")

print("\n" + "="*80)
print(f"STATUS MODULE COMPLETE ({test_accuracy:.1%} accuracy achieved!)")
print("="*80)
print()



STATUS MODELING v2.2 - COMPLETE

┌──────────────────────────────────────────────────────────────────────────────┐
│                          MODELING COMPLETE                                   │
└──────────────────────────────────────────────────────────────────────────────┘

BEST MODEL
--------------------------------------------------------------------------------
Model:           Gradient Boosting
Test Accuracy:   95.1%
F1-macro:        0.951
Train-Test Gap:  0.049 (< 0.10 [OK])

PER-CLASS PERFORMANCE
--------------------------------------------------------------------------------
Beginner:        Precision 97.1%, Recall 100.0%, F1 98.6%
Intermediate:    Precision 91.4%, Recall 94.1%, F1 92.8%
Advanced:        Precision 96.9%, Recall 91.2%, F1 93.9%

OUTPUT FILES
--------------------------------------------------------------------------------
[OK] models/status_best_model_v2.2.pkl
[OK] models/status_model_info_v2.2.json
[OK] models/status_model_comparison_v2.2.json
[OK] visualizati